In [1]:
import pandas as pd
import random
from sqlalchemy import create_engine

from snorkel.labeling import labeling_function

## Load Data

### DataFrame

In [2]:
## load the data by choosing a random 10% of the data
## is the data ordered? am I missing a large chunk due to my partial loading?
# p = 0.01
# responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)

### SQL Engine

In [3]:
# access SQL DB with data
engine = create_engine("sqlite:///../data/moralmachine.db", echo=False)

In [4]:
# demo
responses = pd.read_sql("SELECT * FROM sharedresponses ORDER BY RANDOM() LIMIT 100000", con=engine)
responses.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

In [5]:
# ## For viewing the possible values for each feature
for feature in responses.columns:
    print(feature)
    print(responses[feature].unique())
    print(responses[feature].unique().shape)

ResponseID
['HKQEHx4aZqESmBY8S' '3onZQNgPLe2vsyn9f' 'aLGfC379nQvMMGKnR' ...
 'KhkFWJfcN9mmMqcus' 'uD4GEBYQZbi4mbG34' 'DMSbecfNPBe7oKmaJ']
(99913,)
ExtendedSessionID
['-856676726_1470502651' '794211492_8632120756420629.0'
 '-1755150261_9197295003499718.0' ... '-1697102036_2322956926.0'
 '653309876_8543287825499266.0' '186431870_2178752375268680.0']
(98151,)
UserID
['1470502651' '8632120756420630' '9197295003499720' ... '7574217179790140'
 '8543287825499270' '2178752375268680']
(84098,)
ScenarioOrder
[ 5 10  6  8 11  2  9 13  4  7  1 12  3]
(13,)
Intervention
[0 1]
(2,)
PedPed
[0 1]
(2,)
Barrier
[1 0]
(2,)
CrossingSignal
[0 1 2]
(3,)
AttributeLevel
['Pets' 'Low' 'Fat' 'Old' 'Fit' 'Young' 'Female' 'Less' 'Rand' 'More'
 'Hoomans' 'Male' 'High']
(13,)
ScenarioTypeStrict
['Species' 'Social Status' 'Fitness' 'Age' 'Gender' 'Utilitarian' 'Random']
(7,)
ScenarioType
['Species' 'Social Status' 'Fitness' 'Age' 'Gender' 'Utilitarian' 'Random']
(7,)
DefaultChoice
['Hoomans' 'High' 'Fit' 'Young' 'Ma

In [10]:
# ## What does a single user's session/response look like?
responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    ## see a random user's session
#     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
    ## see the user with nth most responses
    n = 100
    display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]]) 

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,NonDefaultChoice,DefaultChoiceIsOmission,NumberOfCharacters,DiffNumberOFCharacters,Saved,Template,DescriptionShown,LeftHand,UserCountry3,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
69559,Mit9jfwRXWaLPy84C,-1467026630_805057287878687.0,805057287878687,6,0,0,0,2,Rand,Random,Random,,,,3,2,1,Desktop,1,1,AUS,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [7]:
# ## group alternatives pairwise!
# # take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
responses_grouped.shape

(99826, 41)

## Data Exploration

In [9]:
responses.describe()

,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,NumberOfCharacters,DiffNumberOFCharacters,Saved,Man,Woman,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,6.724840,0.499740,0.447900,0.275440,0.604010,3.087860,0.570900,0.502430,0.326760,0.324460,...,0.160170,0.06059,0.108160,0.107640,0.192830,0.192780,0.095810,0.096700,0.172710,0.172980
std,3.773957,0.500002,0.497281,0.446738,0.813799,1.459857,1.150846,0.499997,0.611565,0.610958,...,0.447703,0.25919,0.352679,0.349993,0.525594,0.525394,0.331529,0.333332,0.565159,0.565686
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10.000000,1.000000,1.000000,1.000000,1.000000,4.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,13.000000,1.000000,1.000000,1.000000,2.000000,5.000000,4.000000,1.000000,5.000000,5.000000,...,5.000000,4.00000,4.000000,4.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000


## Snorkel Labeling
Using https://www.snorkel.org/use-cases/01-spam-tutorial

In [27]:
# query random sample of responses grouped by ResponseID; only take responses for which both instances are present
query = """
    --SELECT * FROM (
        SELECT * FROM sharedresponses
            WHERE ScenarioTypeStrict LIKE 'Random'
        ORDER BY RANDOM()
        LIMIT 100000
    --)
    --GROUP BY ResponseID, ExtendedSessionID, UserID
    --HAVING COUNT(ResponseID) > 1
"""
df = pd.read_sql(query, con=engine).groupby(by='ResponseID').filter(lambda g: g.shape[0] > 1).sort_values('ResponseID')
df

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
31018,23k7N4Rm45org4Dnz,-1837873749_1884275148,1884275148,3,1,0,0,1,Rand,Random,...,0,0,1,0,0,0,0,0,0,0
76844,23k7N4Rm45org4Dnz,-1837873749_1884275148,1884275148,3,0,0,1,0,Rand,Random,...,0,0,0,1,0,0,0,0,0,0
85885,26drqtR3HMWQfE6Dn,843359963_3050483523402363.0,3050483523402360,4,0,0,0,2,Rand,Random,...,0,1,0,0,2,0,0,0,0,0
40057,26drqtR3HMWQfE6Dn,843359963_3050483523402363.0,3050483523402360,4,1,0,1,0,Rand,Random,...,0,0,0,1,1,0,0,0,0,0
53629,2T35MoPRr3ECFfEqq,1827679969_6569681646209780.0,6569681646209780,9,0,0,1,0,Rand,Random,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16139,zqmhovsXZhNtxroPv,-1527101226_3189657378630250.0,3189657378630250,10,1,0,1,0,Rand,Random,...,0,0,0,0,1,0,0,0,0,0
43991,ztMiyoAkggaSsEpus,-871545842_1882963901649579.0,1882963901649580,5,1,0,1,0,Rand,Random,...,0,0,0,0,0,0,0,0,0,0
28395,ztMiyoAkggaSsEpus,-871545842_1882963901649579.0,1882963901649580,5,0,0,0,1,Rand,Random,...,0,1,0,0,0,0,1,0,0,0
54616,zwS3ovMRSYbDqWqXA,-2137515174_6468000130095874.0,6468000130095870,4,0,1,0,1,Rand,Random,...,0,0,0,0,0,0,0,0,0,0


Need to have both alternatives in the same tuple, marking them by whether or not they are the intervention - specifically the factors:
> 'NumberOfCharacters', 'DiffNumberOfCharacters', 'Saved', 'Template', 'DescriptionShown',
'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
'Cat'

In [104]:
df[["ResponseID","Intervention","Saved"]]

,ResponseID,Intervention,Saved
31018,23k7N4Rm45org4Dnz,1,1
76844,23k7N4Rm45org4Dnz,0,0
85885,26drqtR3HMWQfE6Dn,0,1
40057,26drqtR3HMWQfE6Dn,1,0
53629,2T35MoPRr3ECFfEqq,0,0
...,...,...,...
16139,zqmhovsXZhNtxroPv,1,0
43991,ztMiyoAkggaSsEpus,1,0
28395,ztMiyoAkggaSsEpus,0,1
54616,zwS3ovMRSYbDqWqXA,0,1


In [127]:
scenario_fields = [
    'ResponseID', 'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'DescriptionShown', 'LeftHand', 'Man', 'Woman', 'Pregnant', 'Stroller', \
    'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', \
    'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat'
]
intervention = df[df['Intervention'] == 1][scenario_fields].set_index('ResponseID')
no_intervention = df[df['Intervention'] == 0][scenario_fields].set_index('ResponseID')
df_joined = intervention.join(no_intervention, lsuffix='_int', rsuffix='_noint', how='inner')
df_joined = df[[col for col in df.columns if col not in scenario_fields or col == 'ResponseID']].set_index('ResponseID').join(df_joined)
df_joined['Intervened'] = (df_joined['Saved_int'] == 1).astype(int)
df_joined = df_joined.drop(axis='columns', labels=['Saved_{}'.format(s) for s in ['int', 'noint']])
df_joined

,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,...,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint,Intervened
ResponseID,,,,,,,,,,,,,,,,,,,,,
23k7N4Rm45org4Dnz,-1837873749_1884275148,1884275148,3,1,0,0,1,Rand,Random,Random,...,0,0,1,0,0,0,0,0,0,1
23k7N4Rm45org4Dnz,-1837873749_1884275148,1884275148,3,0,0,1,0,Rand,Random,Random,...,0,0,1,0,0,0,0,0,0,1
26drqtR3HMWQfE6Dn,843359963_3050483523402363.0,3050483523402360,4,0,0,0,2,Rand,Random,Random,...,1,0,0,2,0,0,0,0,0,0
26drqtR3HMWQfE6Dn,843359963_3050483523402363.0,3050483523402360,4,1,0,1,0,Rand,Random,Random,...,1,0,0,2,0,0,0,0,0,0
2T35MoPRr3ECFfEqq,1827679969_6569681646209780.0,6569681646209780,9,0,0,1,0,Rand,Random,Random,...,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zqmhovsXZhNtxroPv,-1527101226_3189657378630250.0,3189657378630250,10,1,0,1,0,Rand,Random,Random,...,1,0,1,0,0,1,0,1,0,0
ztMiyoAkggaSsEpus,-871545842_1882963901649579.0,1882963901649580,5,1,0,1,0,Rand,Random,Random,...,1,0,0,0,0,1,0,0,0,0
ztMiyoAkggaSsEpus,-871545842_1882963901649579.0,1882963901649580,5,0,0,0,1,Rand,Random,Random,...,1,0,0,0,0,1,0,0,0,0


A standard train test split for testing:

In [195]:
from sklearn.model_selection import train_test_split

X = df_joined.drop(labels=["Intervened"], axis='columns', inplace=False)
y = df_joined["Intervened"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.2, random_state=1)

Some sample labeling functions:

In [196]:
from snorkel.labeling import labeling_function
import numpy as np

"""
:param x: a DataFrame containing all the fields of the input DataFrame for two ethical alternatives
:returns: the index of the alternative to choose (0 or 1); -1 to abstain
"""

def argmax_single(x):
    """
    Accept a single argmax only; if there is a tie, abstain.
    """
    if len(np.argwhere(x == np.max(x))) > 1: return -1
    return x.argmax()

def argmin_single(x):
    """
    Accept a single argmin only; if there is a tie, abstain.
    """
    if len(np.argwhere(x == np.min(x))) > 1: return -1
    return x.argmin()

@labeling_function()
def lf_doctors(x):
    """Favor doctors."""
    num_doctors = np.array([x['MaleDoctor_{}'.format(suffix)] + x['FemaleDoctor_{}'.format(suffix)] for suffix in ['int', 'noint']])
    return argmax_single(num_doctors)

@labeling_function()
def lf_utilitarian(x):
    """Favor saving more lives."""
    max_characters = np.array([x['NumberOfCharacters_{}'.format(suffix)] for suffix in ['int', 'noint']])
    return argmax_single(max_characters)

In [198]:
from snorkel.labeling import PandasLFApplier

lfs = [lf_doctors, lf_utilitarian]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=X_train)
L_dev = applier.apply(df=X_dev)
L_train














  0%|          | 0/1072 [00:00<?, ?it/s]












100%|██████████| 1072/1072 [00:00<00:00, 7467.62it/s][A













100%|██████████| 68/68 [00:00<00:00, 6015.62it/s]A


array([[ 1, -1],
       [ 0,  0],
       [-1,  1],
       ...,
       [-1,  1],
       [ 0,  0],
       [ 1,  1]])

In [199]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_doctors,0,"[0, 1]",0.396455,0.318097,0.114739
lf_utilitarian,1,"[0, 1]",0.789179,0.318097,0.114739


In [202]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=y_dev.values)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_doctors,0,"[0, 1]",0.397059,0.338235,0.132353,12,15,0.444444
lf_utilitarian,1,"[0, 1]",0.735294,0.338235,0.132353,14,36,0.280000


In [205]:
from snorkel.analysis import get_label_buckets

# misclassified - labeled intervention, actually not
buckets = get_label_buckets(y_dev, L_dev[:, 1])
X_dev.iloc[buckets[0,1]]

,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,...,LargeMan_noint,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint
ResponseID,,,,,,,,,,,,,,,,,,,,,
TcKnax2oaQy5xzGmH,-702950869_7386194413255684.0,7386194413255680,7,1,0,0,1,Rand,Random,Random,...,0,0,0,0,0,0,1,0,0,0
e43FbT4H6vfpy9K3E,-1156582688_3788750884.0,3788750884,10,0,0,1,0,Rand,Random,Random,...,0,0,0,0,0,0,0,1,0,0
8fgSYAYPQ6WujKTXy,-1623983207_1973495769,1973495769,2,1,0,1,0,Rand,Random,Random,...,0,0,1,1,0,0,1,0,0,0
ZdsShHpzitcQGRszS,-1343331337_2156404150.0,2156404150,1,1,0,1,0,Rand,Random,Random,...,0,0,0,1,0,0,0,1,0,1
uqPGBw3tuCxDSLQ9w,-1673863700_2903735528542186.0,2903735528542190,13,0,0,0,1,Rand,Random,Random,...,1,1,0,1,0,0,0,0,0,1
gnb7pyLsK9aoBtpJW,1968508180_7075821972565746.0,7075821972565750,3,0,0,0,2,Rand,Random,Random,...,0,0,0,0,0,0,1,0,0,0
RiJwgjnda9ATrBuD7,-495798711_4059499457.0,4059499457,4,1,0,1,0,Rand,Random,Random,...,0,0,0,0,1,0,0,0,0,0
46pgYQmdoRkD7RMLs,-1470494871_7306205927734435.0,7306205927734440,7,1,0,1,0,Rand,Random,Random,...,0,0,0,0,0,0,0,0,0,0
NNhTf8gtH7MNZFRsk,685464590_6067949408670390.0,6067949408670390,12,1,0,0,2,Rand,Random,Random,...,0,1,0,1,0,0,1,0,0,0


In [208]:
## TODO: check this - probably buggy
# false positives? (intervene when not supposed to?)
# this is where doctors voted intervene
X_train.iloc[L_train[:, 1] == 1].sample(10, random_state=1)
# this is where doctors abstained, but utilitarian voted
X_train.iloc[buckets[(0, -1)]].sample(10, random_state=1)

,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,...,LargeMan_noint,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint
ResponseID,,,,,,,,,,,,,,,,,,,,,
GuyYzEA3mB8F8cBSk,-1520329119_7902954156989065.0,7902954156989070,11,1,0,0,2,Rand,Random,Random,...,0,1,0,0,0,0,0,0,0,0
Zv4JN5BR6wuAYBCes,916906793_6047230056112887.0,6047230056112890,3,0,0,0,1,Rand,Random,Random,...,0,0,0,0,1,0,0,0,0,1
35ZBgpj6k223Za3oE,76165960_4520132560267611.0,4520132560267610,2,0,0,0,2,Rand,Random,Random,...,0,0,0,0,0,0,0,0,0,0
PeHrBcep4xPtyzw3c,-1240613373_505571791505905.0,505571791505905,11,0,0,0,1,Rand,Random,Random,...,1,1,0,0,0,0,0,0,1,0
8qMXugegAdJWCGabL,515347274_504148777,504148777,8,0,0,1,0,Rand,Random,Random,...,0,0,0,0,2,0,0,0,0,0
8qMXugegAdJWCGabL,515347274_504148777,504148777,8,1,0,0,2,Rand,Random,Random,...,0,0,0,0,2,0,0,0,0,0
FTRfFCaoo4NCqAggP,-1665125873_1889168559,1889168559,7,0,0,0,1,Rand,Random,Random,...,0,0,0,0,0,0,0,1,0,0
uP79WHftPdtShgpFY,-467338067_2199892323.0,2199892323,2,1,0,1,0,Rand,Random,Random,...,0,0,0,0,0,0,0,0,0,0
8DH9Ej2hTLL7RaWQS,156367721_2191860122.0,2191860122,7,0,0,0,1,Rand,Random,Random,...,0,0,0,0,0,0,0,1,1,0
